In [1]:
# Instalo las librerías necesarias para usar la API de Fintoc y poder generar reportes en CSV
!pip -q install fintoc python-dotenv pandas


In [110]:
# Cargo la API Key de Fintoc como variable de entorno para poder autenticarme con la API en modo test
import os

#la borre para github


In [3]:
# Creo una carpeta para guardar las llaves JWS que se usarán para firmar las transferencias
!mkdir -p keys
!ls -l


total 8
drwxr-xr-x 2 root root 4096 Jan 12 21:31 keys
drwxr-xr-x 1 root root 4096 Dec 11 14:34 sample_data


In [4]:
# Genero el par de llaves JWS (privada y pública) , La privada se queda local y la pública se sube al dashboard de Fintoc
!openssl genrsa -out keys/jws_private_key.pem 2048
!openssl rsa -in keys/jws_private_key.pem -pubout -out keys/jws_public_key.pem
!ls -l keys


writing RSA key
total 8
-rw------- 1 root root 1704 Jan 12 21:31 jws_private_key.pem
-rw-r--r-- 1 root root  451 Jan 12 21:31 jws_public_key.pem


In [6]:
# Descargo la llave pública para subirla al dashboard de Fintoc
from google.colab import files
files.download("keys/jws_public_key.pem")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Vuelvo a generar las llaves JWS usando el formato exacto que pide la documentación de Fintoc para firmar transfers
!rm -rf keys
!mkdir -p keys
!openssl genrsa -out keys/private_key.pem 2048
!openssl rsa -in keys/private_key.pem -outform PEM -pubout -out keys/public_key.pem.pub
!ls -l keys


writing RSA key
total 8
-rw------- 1 root root 1704 Jan 12 21:50 private_key.pem
-rw-r--r-- 1 root root  451 Jan 12 21:50 public_key.pem.pub


In [8]:
# Imprimo la llave pública en texto para copiarla y pegarla directamente en el dashboard de Fintoc
print("=== PUBLIC KEY (para pegar en Fintoc) ===")
with open("keys/public_key.pem.pub", "r") as f:
    public_key_text = f.read()

print(public_key_text)
print("=== FIN ===")


=== PUBLIC KEY (para pegar en Fintoc) ===
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAtZC5sl4J8RDfVjD/vc7c
NgEkmt8Qp/lz2hEXrES2zIresO6cL92OC0XyvcwYv/f9N+qsqu5kqKzxlX7D/pdG
oqcXyytwAVXmh894loroXRSexw9RozXTAhZNFmn/GRGLcN8biWGQMmDWKHDio4tn
yzTcxLDOT+J6bZQeYMe9kL5XlXGMnpZ0zUEKFhz9sM/hwL3xHX2CXJO7g5DncnLf
QwCQ5N9vCLsnumzAkWqyrPMC2+6LS0GyMNR0ZmJXu/96/PXNsZihW7PVI/LtJbTm
kVWpw5HKKfW10Z3pCY4yxu9hVFovIOrIZjHQ06+tsJAD91sqR4g9aoBsPnvasVuX
SwIDAQAB
-----END PUBLIC KEY-----

=== FIN ===


In [109]:
# Inicializo el cliente de Fintoc usando la API Key y una llave privada JWS.
# La llave privada se genera localmente y NO se sube al repositorio.
# En este notebook se asume que el archivo existe en el path indicado.
from fintoc import Fintoc
import os

client = Fintoc(
    os.environ["FINTOC_SECRET_KEY"],
    jws_private_key="keys/private_key.pem"
)

accounts = list(client.v2.accounts.list())
print(accounts)


In [71]:
# Exploro la cuenta para entender qué información entrega Fintoc
# (esto es exploratorio / debug para conocer los campos disponibles)
accounts = list(client.v2.accounts.list())

print("Cantidad de cuentas:", len(accounts))
a = accounts[0]

# Mostrar info básica (forma 1)
try:
    print("DICT:", a.__dict__)
except Exception as e:
    print("No pude imprimir __dict__:", e)

# Mostrar campos típicos (forma 2)
for field in ["id", "currency", "available_balance", "balance", "type", "name"]:
    if hasattr(a, field):
        print(field, "=", getattr(a, field))


Cantidad de cuentas: 1
DICT: {'_client': <fintoc.client.Client object at 0x7db568d2c4a0>, '_handlers': {'update': <bound method ManagerMixin.post_update_handler of <fintoc.managers.v2.accounts_manager.AccountsManager object at 0x7db5572cf230>>, 'delete': <bound method ManagerMixin.post_delete_handler of <fintoc.managers.v2.accounts_manager.AccountsManager object at 0x7db5572cf230>>}, '_methods': ['list', 'get', 'create', 'update'], '_path': '/v2/accounts', '_attributes': ['id', 'object', 'mode', 'root_account_number', 'is_root', 'root_account_number_id', 'available_balance', 'currency', 'entity', 'description', 'status'], 'id': 'acc_388ZcnIcTbLP91rBLLAc8zS9Fog', 'object': 'account', 'mode': 'test', 'root_account_number': '17681832392936', 'is_root': True, 'root_account_number_id': 'acno_388Zcn8sbU94DypTWErqqrrDwU1', 'available_balance': 999000, 'currency': 'CLP', 'entity': <fintoc.resources.v2.entity.Entity object at 0x7db545c8d9a0>, 'description': None, 'status': 'active', '_Account__

In [72]:
# Me quedo con el account_id de la cuenta de origen y reviso el balance disponible antes de hacer transferencias
accounts = list(client.v2.accounts.list())
a = accounts[0]
print("account_id:", a.id)
print("available_balance:", a.available_balance)


account_id: acc_388ZcnIcTbLP91rBLLAc8zS9Fog
available_balance: 999000


In [73]:
# Llamada directa a la API REST para listar instituciones
# Esta celda es solo de debug: sirve para entender qué bancos existen y qué institution_id usar en el counterparty
import os, requests

API_KEY = os.environ["FINTOC_SECRET_KEY"]

url = "https://api.fintoc.com/v1/institutions"
resp = requests.get(url, headers={"Authorization": API_KEY})

print("URL:", url)
print("status:", resp.status_code)
print("raw text (primeros 300 chars):", resp.text[:300])


URL: https://api.fintoc.com/v1/institutions
status: 200
raw text (primeros 300 chars): [{"id":"cl_banco_deutsche","name":"Banco Deutsche","country":"cl","products":[],"object_name":"institution","type":"bank"},{"id":"mx_banco_fincomun","name":"Banco Fincomun","country":"mx","products":[],"object_name":"institution","type":"bank"},{"id":"mx_mercado_pago","name":"Mercado Pago","country"


In [74]:
# Convierto la respuesta de institutions a JSON
# y reviso rápido que sea una lista y qué trae

import json

data = resp.json()
print("tipo:", type(data))
print("cantidad:", len(data) if isinstance(data, list) else "no es lista")

# imprime 10 bancos (id + name)
if isinstance(data, list):
    for inst in data[:10]:
        print(inst.get("id"), "-", inst.get("name"))
else:
    print("Respuesta no es lista. JSON:", json.dumps(data, indent=2, ensure_ascii=False))


tipo: <class 'list'>
cantidad: 90
cl_banco_deutsche - Banco Deutsche
mx_banco_fincomun - Banco Fincomun
mx_mercado_pago - Mercado Pago
mx_banco_banxico - Banco Banxico
mx_banco_arcus - Arcus
mx_asp - ASP
mx_nu - Nu Mexico
mx_klar - Klar
mx_cuenca - Cuenca
cl_fintual - Fintual


In [75]:
# Filtro solo bancos de Chile para sacar un institution_id válido

data = resp.json()

cl_banks = [i for i in data if i.get("country") == "cl" and i.get("type") == "bank"]

print("Bancos de Chile:", len(cl_banks))
for inst in cl_banks:
    print(inst["id"], "-", inst["name"])


Bancos de Chile: 29
cl_banco_deutsche - Banco Deutsche
cl_fintual - Fintual
cl_prepago_la_polar - Prepago La Polar
cl_banco_internacional - Banco Internacional
cl_banco_estado - Banco Estado
cl_banco_scotiabank - Banco Scotiabank
cl_banco_bci - Banco BCI
cl_banco_bci_360 - Banco BCI
cl_banco_corpbanca - Banco Corpbanca
cl_banco_bice - Banco BICE
cl_banco_hsbc - HSBC
cl_banco_itau - Banco Itaú
cl_banco_security - Banco Security
cl_banco_falabella - Banco Falabella
cl_banco_ripley - Banco Ripley
cl_banco_consorcio - Banco Consorcio
cl_transbank - Transbank
cl_banco_bbva - Banco BBVA
cl_banco_coopeuch - Coopeuch / Dale
cl_prepago_los_heroes - Prepago Los Héroes
cl_tenpo - Tenpo
cl_global66 - Global 66
cl_copec_pay - Copec Pay
cl_prex - Prex
cl_mercado_pago - Mercado Pago
cl_banco_de_chile - Banco de Chile
cl_banco_jp_morgan - JP Morgan
cl_banco_santander - Banco Santander
cl_tapp_caja_los_andes - TAPP Caja los Andes


In [76]:
# Prueba puntual: consulto una transferencia por id para ver qué campos devuelve (status, amount, fechas, etc.)
transfer_id = "tr_38B8Fc3OQOxUrh51N3fkp2CSAGL"

t = client.v2.transfers.get(transfer_id)
print("id:", t.id)
print("status:", getattr(t, "status", None))
print("amount:", getattr(t, "amount", None))
print("created_at:", getattr(t, "created_at", None))


id: tr_38B8Fc3OQOxUrh51N3fkp2CSAGL
status: succeeded
amount: 1000
created_at: None


In [77]:
# Parto el monto total en varias transferencias, respetando el máximo de Chile
TOTAL = 500_000_000
MAX_PER_TRANSFER = 7_000_000

amounts = []
remaining = TOTAL
while remaining > 0:
    chunk = min(MAX_PER_TRANSFER, remaining)
    amounts.append(chunk)
    remaining -= chunk

print("Cantidad de transferencias:", len(amounts))
print("Primeras 5:", amounts[:5])
print("Última:", amounts[-1])
print("Suma total:", sum(amounts))


Cantidad de transferencias: 72
Primeras 5: [7000000, 7000000, 7000000, 7000000, 7000000]
Última: 3000000
Suma total: 500000000


In [78]:
# Guardo el CSV con el resultado de la creación
df.to_csv("transfers_report_create_v2.csv", index=False)


In [79]:
# Reviso qué archivos quedaron guardados (los CSV)

!ls -lh
!ls -lh *.csv || true


total 76K
drwxr-xr-x 2 root root 4.0K Jan 12 21:50 keys
drwxr-xr-x 1 root root 4.0K Dec 11 14:34 sample_data
-rw-r--r-- 1 root root    1 Jan 12 23:50 transfers_report_create.csv
-rw-r--r-- 1 root root 8.3K Jan 13 00:09 transfers_report_create_run_1768262415.csv
-rw-r--r-- 1 root root 9.8K Jan 13 01:15 transfers_report_create_v2.csv
-rw-r--r-- 1 root root 6.4K Jan 13 01:05 transfers_report_final_72.csv
-rw-r--r-- 1 root root 3.4K Jan 12 23:56 transfers_report_final.csv
-rw-r--r-- 1 root root 9.5K Jan 13 00:24 transfers_report_final_run_1768262415_clean.csv
-rw-r--r-- 1 root root 9.8K Jan 13 00:21 transfers_report_final_run_1768262415.csv
-rw-r--r-- 1 root root  178 Jan 13 00:24 transfers_report_summary_run_1768262415.json
-rw-r--r-- 1 root root    1 Jan 12 23:50 transfers_report_create.csv
-rw-r--r-- 1 root root 8.3K Jan 13 00:09 transfers_report_create_run_1768262415.csv
-rw-r--r-- 1 root root 9.8K Jan 13 01:15 transfers_report_create_v2.csv
-rw-r--r-- 1 root root 6.4K Jan 13 01:05 tra

In [80]:
# Backup: si df existe lo guardo; si no, aviso (esto por si se reiniciaba la sesión o se perdía df)
import pandas as pd

# Si df existe (del loop), lo guardamos ahora mismo
print("df existe?", "df" in globals())
if "df" in globals():
    print("df shape:", df.shape)
    display(df.head())
    df.to_csv("transfers_report_create_v2.csv", index=False)
    print("Guardado: transfers_report_create_v2.csv")
else:
    print("No encuentro df en memoria. Vamos a reconstruirlo desde records.")


df existe? True
df shape: (72, 11)


,comment,n,transfer_id,amount,currency,status,transaction_date,post_date,reference_id,tracking_key,receipt_url
0,fintoc_case_run_1768262415_batch_1,1,tr_38BBAH5smVf1c5tgQDfohxugKjx,7000000,CLP,succeeded,2026-01-13T00:09:05Z,2026-01-13T00:00:00Z,None,None,None
1,fintoc_case_run_1768262415_batch_2,2,tr_38BBAMhBgZT9v5HOdCdrEamqKIn,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None
2,fintoc_case_run_1768262415_batch_3,3,tr_38BBAOOh9MftbQEcp88zxnqlW2l,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None
3,fintoc_case_run_1768262415_batch_4,4,tr_38BBAS1EV9gMw0dq52Q2JCXkQss,7000000,CLP,succeeded,2026-01-13T00:09:05Z,2026-01-13T00:00:00Z,None,None,None
4,fintoc_case_run_1768262415_batch_5,5,tr_38BBAWcOKGDgMGBdkhYVZcgpIZT,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None


Guardado: transfers_report_create_v2.csv


In [82]:
# Creo las 72 transferencias usando la API, con comment único para trazabilidad y guardo un reporte local con el resultado de creación (ok/error + transfer_id)

import time
import pandas as pd

TOTAL = 500_000_000
MAX_PER_TRANSFER = 7_000_000

amounts = []
remaining = TOTAL
while remaining > 0:
    chunk = min(MAX_PER_TRANSFER, remaining)
    amounts.append(chunk)
    remaining -= chunk

account_id = a.id
institution_id = "cl_banco_de_chile"

counterparty_base = {
    "holder_id": "11111111-1",
    "holder_name": "Destinatario Prueba",
    "account_number": "12345678",
    "institution_id": institution_id,
    "type": "checking"
}

records = []

for i, amt in enumerate(amounts, start=1):
    try:
        tr = client.v2.transfers.create(
            account_id=account_id,
            amount=amt,
            currency="CLP",
            counterparty=counterparty_base,
            comment=f"fintoc_case_batch_{i}_{int(time.time())}"
        )
        tr_id = getattr(tr, "id", None)
        print(f"[{i}/{len(amounts)}] OK amount={amt} id={tr_id}")

        records.append({
            "n": i,
            "amount": amt,
            "transfer_id": tr_id,
            "created_at_local": time.strftime("%Y-%m-%d %H:%M:%S"),
            "comment": f"fintoc_case_batch_{i}",
            "create_result": "ok",
            "error": None
        })

    except Exception as e:
        print(f"[{i}/{len(amounts)}] ERROR amount={amt} -> {e}")
        records.append({
            "n": i,
            "amount": amt,
            "transfer_id": None,
            "created_at_local": time.strftime("%Y-%m-%d %H:%M:%S"),
            "comment": f"fintoc_case_batch_{i}",
            "create_result": "error",
            "error": str(e)
        })

    time.sleep(0.2)

df = pd.DataFrame(records)
df.to_csv("transfers_report_create_v2.csv", index=False)
print("Guardado transfers_report_create_v2.csv con shape:", df.shape)
df.head()


[1/72] OK amount=7000000 id=tr_38BJPGqCydW2sCAQzbtilP31SXg
[2/72] OK amount=7000000 id=tr_38BJPIso8KhyOx2QTrjlhDTkObK
[3/72] OK amount=7000000 id=tr_38BJPFO4YGUo4jBXQQMtEGPVnDD
[4/72] OK amount=7000000 id=tr_38BJPRWjjMxHwDsuPR7SYn1i7Ak
[5/72] OK amount=7000000 id=tr_38BJPKGqrvMePsfhAe7ph6oqH5t
[6/72] OK amount=7000000 id=tr_38BJPM9TN1gvUqPA4hpURWrNMbB
[7/72] OK amount=7000000 id=tr_38BJPSUaEA7csmOq84M1bcZF3FU
[8/72] OK amount=7000000 id=tr_38BJPWyEyWrXFO8mCfMWZ1vxaUc
[9/72] OK amount=7000000 id=tr_38BJPSSIVvj334oylZzVdQbKDAh
[10/72] OK amount=7000000 id=tr_38BJPerEKAZVbaAqCx5A0pP9Ezn
[11/72] OK amount=7000000 id=tr_38BJPeF3IIn13TDrqZE85FNkffQ
[12/72] OK amount=7000000 id=tr_38BJPf134bL1U1yMGe2PoPvyv9H
[13/72] OK amount=7000000 id=tr_38BJPjZ1CsWSwTsbbVzaOI2JXH3
[14/72] OK amount=7000000 id=tr_38BJPmHHrPBjlkoOmqzOFdWkgVn
[15/72] OK amount=7000000 id=tr_38BJPoWNDZA8zhuL0icY9fcp5t2
[16/72] OK amount=7000000 id=tr_38BJPvpoTaA1Jt1RXR22FbYvN9f
[17/72] OK amount=7000000 id=tr_38BJPqbGO3kLvqpgl

,n,amount,transfer_id,created_at_local,comment,create_result,error
0,1,7000000,tr_38BJPGqCydW2sCAQzbtilP31SXg,2026-01-13 01:16:44,fintoc_case_batch_1,ok,None
1,2,7000000,tr_38BJPIso8KhyOx2QTrjlhDTkObK,2026-01-13 01:16:44,fintoc_case_batch_2,ok,None
2,3,7000000,tr_38BJPFO4YGUo4jBXQQMtEGPVnDD,2026-01-13 01:16:44,fintoc_case_batch_3,ok,None
3,4,7000000,tr_38BJPRWjjMxHwDsuPR7SYn1i7Ak,2026-01-13 01:16:45,fintoc_case_batch_4,ok,None
4,5,7000000,tr_38BJPKGqrvMePsfhAe7ph6oqH5t,2026-01-13 01:16:45,fintoc_case_batch_5,ok,None


In [84]:
# Consulto el estado de cada transferencia creada (usando su transfer_id) y genero el reporte final con status para cada una
import pandas as pd
import time

df = pd.read_csv("transfers_report_create_v2.csv")

statuses = []
for i, row in df.iterrows():
    tr_id = row["transfer_id"]

    try:
        t = client.v2.transfers.get(tr_id)
        statuses.append(getattr(t, "status", None))
    except Exception as e:
        statuses.append(f"error: {str(e)[:120]}")

    # pausa pequeña para no saturar
    time.sleep(0.15)

df["status"] = statuses

# resumen útil
print(df["status"].value_counts(dropna=False))

df.to_csv("transfers_report_final.csv", index=False)
df.head()


status
succeeded    66
pending       6
Name: count, dtype: int64


,n,amount,transfer_id,created_at_local,comment,create_result,error,status
0,1,7000000,tr_38BJPGqCydW2sCAQzbtilP31SXg,2026-01-13 01:16:44,fintoc_case_batch_1,ok,NaN,succeeded
1,2,7000000,tr_38BJPIso8KhyOx2QTrjlhDTkObK,2026-01-13 01:16:44,fintoc_case_batch_2,ok,NaN,succeeded
2,3,7000000,tr_38BJPFO4YGUo4jBXQQMtEGPVnDD,2026-01-13 01:16:44,fintoc_case_batch_3,ok,NaN,succeeded
3,4,7000000,tr_38BJPRWjjMxHwDsuPR7SYn1i7Ak,2026-01-13 01:16:45,fintoc_case_batch_4,ok,NaN,succeeded
4,5,7000000,tr_38BJPKGqrvMePsfhAe7ph6oqH5t,2026-01-13 01:16:45,fintoc_case_batch_5,ok,NaN,succeeded


In [85]:
# Tomo el CSV de creación, saco las filas que no tienen transfer_id y consulto el status solo de las que realmente existen
import pandas as pd
import time

df = pd.read_csv("transfers_report_create_v2.csv")

# 1) limpiar: dejar solo filas con transfer_id real
df_ok = df[df["transfer_id"].notna()].copy()
print("Total filas:", len(df), "| con transfer_id:", len(df_ok), "| sin transfer_id:", len(df) - len(df_ok))

# 2) consultar status solo para las que existen
statuses = []
for tr_id in df_ok["transfer_id"]:
    try:
        t = client.v2.transfers.get(tr_id)
        statuses.append(getattr(t, "status", None))
    except Exception as e:
        statuses.append(f"error: {str(e)[:120]}")
    time.sleep(0.12)

df_ok["status"] = statuses

# 3) resumen bonito
print("\nResumen status:")
print(df_ok["status"].value_counts(dropna=False))

# 4) guardar CSV final limpio
df_ok.to_csv("transfers_report_final.csv", index=False)
df_ok.head()


Total filas: 72 | con transfer_id: 72 | sin transfer_id: 0

Resumen status:
status
succeeded    72
Name: count, dtype: int64


,n,amount,transfer_id,created_at_local,comment,create_result,error,status
0,1,7000000,tr_38BJPGqCydW2sCAQzbtilP31SXg,2026-01-13 01:16:44,fintoc_case_batch_1,ok,NaN,succeeded
1,2,7000000,tr_38BJPIso8KhyOx2QTrjlhDTkObK,2026-01-13 01:16:44,fintoc_case_batch_2,ok,NaN,succeeded
2,3,7000000,tr_38BJPFO4YGUo4jBXQQMtEGPVnDD,2026-01-13 01:16:44,fintoc_case_batch_3,ok,NaN,succeeded
3,4,7000000,tr_38BJPRWjjMxHwDsuPR7SYn1i7Ak,2026-01-13 01:16:45,fintoc_case_batch_4,ok,NaN,succeeded
4,5,7000000,tr_38BJPKGqrvMePsfhAe7ph6oqH5t,2026-01-13 01:16:45,fintoc_case_batch_5,ok,NaN,succeeded


In [86]:
# Llamada directa a la API v2/transfers para ver la estructura real del JSON
# (esto para entender qué campos trae y cómo viene la lista)
import os, requests, json

API_KEY = os.environ["FINTOC_SECRET_KEY"]

url = "https://api.fintoc.com/v2/transfers"
r = requests.get(url, headers={"Authorization": API_KEY})

print("status:", r.status_code)
data = r.json()
print("tipo:", type(data))
print("muestra:", json.dumps(data[:2], indent=2, ensure_ascii=False) if isinstance(data, list) else json.dumps(data, indent=2, ensure_ascii=False))


status: 200
tipo: <class 'list'>
muestra: [
  {
    "object": "transfer",
    "id": "tr_38BJS6KaKAzrcJklAtp68T041UO",
    "amount": 3000000,
    "currency": "CLP",
    "direction": "outbound",
    "status": "succeeded",
    "transaction_date": "2026-01-13T01:17:23Z",
    "post_date": "2026-01-13T00:00:00Z",
    "comment": "fintoc_case_batch_72_1768267027",
    "reference_id": null,
    "tracking_key": null,
    "receipt_url": null,
    "mode": "test",
    "counterparty": {
      "holder_id": "111111111",
      "holder_name": "Destinatario Prueba",
      "account_number": "12345678",
      "account_type": null,
      "institution": {
        "id": "cl_banco_de_chile",
        "name": "Banco De Chile",
        "country": "cl"
      }
    },
    "account_number": {
      "id": "acno_388Zcn8sbU94DypTWErqqrrDwU1",
      "account_id": "acc_388ZcnIcTbLP91rBLLAc8zS9Fog",
      "number": "17681832392936",
      "created_at": "2026-01-12T02:00:39Z",
      "updated_at": "2026-01-12T02:00:41Z",
  

In [87]:
# Filtro las transfers usando el comment como “marca” y las ordeno por número de batch

# filtrar por nuestro prefijo de comment
mine = [t for t in data if isinstance(t, dict) and str(t.get("comment","")).startswith("fintoc_case_batch_")]

print("Encontradas con nuestro comment:", len(mine))

# ordenar por número de batch si viene en el comment
def get_n(t):
    # comment ejemplo: fintoc_case_batch_12_170...
    parts = str(t.get("comment","")).split("_")
    # ['fintoc','case','batch','12','170...']
    return int(parts[3]) if len(parts) > 3 and parts[3].isdigit() else 10**9

mine = sorted(mine, key=get_n)

# mostrar 5 primeras
for t in mine[:5]:
    print(t.get("id"), t.get("amount"), t.get("status"), t.get("comment"))


Encontradas con nuestro comment: 30
tr_38BJR1MN25kdYz7K4HSu1CVBVm7 7000000 succeeded fintoc_case_batch_43_1768267017
tr_38BJQym1C1pz48TV1943LOq9jha 7000000 succeeded fintoc_case_batch_44_1768267018
tr_38BJR3aVIUR3VnYisaHEpDKGERD 7000000 succeeded fintoc_case_batch_45_1768267018
tr_38BJQzYJ7lnSbG0wArDU56Z57vi 7000000 succeeded fintoc_case_batch_46_1768267018
tr_38BJR9wYzezZ81TqOlfHwcTYlSX 7000000 succeeded fintoc_case_batch_47_1768267019


In [88]:
# Armo un dataframe con las transfers filtradas y lo guardo como CSV final (versión simple)
import pandas as pd

df_final = pd.DataFrame([{
    "transfer_id": t.get("id"),
    "amount": t.get("amount"),
    "status": t.get("status"),
    "comment": t.get("comment"),
    "created_at": t.get("created_at"),
    "currency": t.get("currency"),
} for t in mine])

print("shape:", df_final.shape)
df_final.to_csv("transfers_report_final_72.csv", index=False)
df_final.head()


shape: (30, 6)


,transfer_id,amount,status,comment,created_at,currency
0,tr_38BJR1MN25kdYz7K4HSu1CVBVm7,7000000,succeeded,fintoc_case_batch_43_1768267017,None,CLP
1,tr_38BJQym1C1pz48TV1943LOq9jha,7000000,succeeded,fintoc_case_batch_44_1768267018,None,CLP
2,tr_38BJR3aVIUR3VnYisaHEpDKGERD,7000000,succeeded,fintoc_case_batch_45_1768267018,None,CLP
3,tr_38BJQzYJ7lnSbG0wArDU56Z57vi,7000000,succeeded,fintoc_case_batch_46_1768267018,None,CLP
4,tr_38BJR9wYzezZ81TqOlfHwcTYlSX,7000000,succeeded,fintoc_case_batch_47_1768267019,None,CLP


In [89]:
# Función para traer TODAS las transfers usando paginación
# (si no pagino, me quedo con una parte y el conteo sale mal)
import os, requests

API_KEY = os.environ["FINTOC_SECRET_KEY"]

def list_all_transfers():
    all_transfers = []
    starting_after = None

    while True:
        params = {}
        if starting_after:
            params["starting_after"] = starting_after

        r = requests.get(
            "https://api.fintoc.com/v2/transfers",
            headers={"Authorization": API_KEY},
            params=params
        )

        data = r.json()
        if not isinstance(data, list) or len(data) == 0:
            break

        all_transfers.extend(data)
        starting_after = data[-1]["id"]  # 👈 clave de paginación

        if len(data) < 30:
            break

    return all_transfers


In [90]:
# Uso la paginación para traer todo y filtro solo lo mío por comment
# (Paginación es cuando una API no te entrega todos los resultados de una sola vez, sino que te los devuelve por partes (páginas)).
all_transfers = list_all_transfers()
print("Total transfers traídas:", len(all_transfers))

mine = [
    t for t in all_transfers
    if isinstance(t, dict)
    and str(t.get("comment", "")).startswith("fintoc_case_batch_")
]

print("Transfers del caso:", len(mine))

# ordenarlas por número de batch
def get_batch(t):
    try:
        return int(t["comment"].split("_")[3])
    except:
        return 9999

mine = sorted(mine, key=get_batch)

# mostrar primeras 5
for t in mine[:5]:
    print(t["id"], t["amount"], t["status"], t["comment"])


Total transfers traídas: 294
Transfers del caso: 179
tr_38BJPGqCydW2sCAQzbtilP31SXg 7000000 succeeded fintoc_case_batch_1_1768267003
tr_38B9M4yJ53z1SQNsOGNHKbodbwZ 7000000 succeeded fintoc_case_batch_1_1768262045
tr_38B8raNmqV8lQKSiVWD5HJjt7n6 7000000 succeeded fintoc_case_batch_1_1768261802
tr_38BJPIso8KhyOx2QTrjlhDTkObK 7000000 succeeded fintoc_case_batch_2_1768267004
tr_38B9M7AZyhTjn5Ryt4uJVcLozOe 7000000 succeeded fintoc_case_batch_2_1768262045


In [91]:
# Armo el CSV final usando lo filtrado desde la API completa (con paginación)
import pandas as pd

df_final = pd.DataFrame([{
    "transfer_id": t["id"],
    "amount": t["amount"],
    "status": t["status"],
    "comment": t["comment"],
    "created_at": t.get("created_at"),
    "currency": t["currency"],
} for t in mine])

print("Shape final:", df_final.shape)
df_final.to_csv("transfers_report_final_72.csv", index=False)
df_final.head()


Shape final: (179, 6)


,transfer_id,amount,status,comment,created_at,currency
0,tr_38BJPGqCydW2sCAQzbtilP31SXg,7000000,succeeded,fintoc_case_batch_1_1768267003,None,CLP
1,tr_38B9M4yJ53z1SQNsOGNHKbodbwZ,7000000,succeeded,fintoc_case_batch_1_1768262045,None,CLP
2,tr_38B8raNmqV8lQKSiVWD5HJjt7n6,7000000,succeeded,fintoc_case_batch_1_1768261802,None,CLP
3,tr_38BJPIso8KhyOx2QTrjlhDTkObK,7000000,succeeded,fintoc_case_batch_2_1768267004,None,CLP
4,tr_38B9M7AZyhTjn5Ryt4uJVcLozOe,7000000,succeeded,fintoc_case_batch_2_1768262045,None,CLP


In [92]:
# Creo un identificador único del “run” para filtrar después SOLO estas 72 transfers y no mezclar con pruebas antiguas
RUN_ID = str(int(time.time()))
print("RUN_ID:", RUN_ID)


RUN_ID: 1768267142


In [96]:
print("RUN_ID actual:", RUN_ID)
print("Prefix buscado:", f"fintoc_case_run_{RUN_ID}_batch_")


RUN_ID actual: 1768267142
Prefix buscado: fintoc_case_run_1768267142_batch_


In [97]:
all_transfers = list_all_transfers()
print("Total transfers traídas:", len(all_transfers))

# muestra 5 comments cualquiera (si existen)
comments = [t.get("comment") for t in all_transfers if isinstance(t, dict) and t.get("comment")]
print("Ejemplos de comments:")
for c in comments[:5]:
    print("-", c)


Total transfers traídas: 294
Ejemplos de comments:
- fintoc_case_batch_72_1768267027
- fintoc_case_batch_71_1768267027
- fintoc_case_batch_70_1768267026
- fintoc_case_batch_69_1768267026
- fintoc_case_batch_68_1768267026


In [98]:
import re
all_transfers = list_all_transfers()

pattern = re.compile(r"^fintoc_case_run_(\d+)_batch_(\d+)$")

runs = {}
for t in all_transfers:
    c = t.get("comment", "")
    m = pattern.match(str(c))
    if m:
        rid = m.group(1)
        runs[rid] = runs.get(rid, 0) + 1

print("RUN_IDs encontrados (run_id -> cantidad transfers):")
for rid, cnt in sorted(runs.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(rid, "->", cnt)


RUN_IDs encontrados (run_id -> cantidad transfers):
1768262415 -> 73


In [99]:
def build_df_for_run(all_transfers, prefix):
    mine = [
        t for t in all_transfers
        if isinstance(t, dict) and str(t.get("comment", "")).startswith(prefix)
    ]

    # 1) agrupar por comment (clave del batch)
    by_comment = {}
    for t in mine:
        c = t.get("comment")
        if not c:
            continue
        by_comment.setdefault(c, []).append(t)

    # 2) elegir 1 por comment (dedupe real)
    dedup = []
    for c, rows in by_comment.items():
        best = pick_best_row(rows)
        if best:
            dedup.append(best)

    # 3) DataFrame final (1 fila por batch/comment)
    df = pd.DataFrame([{
        "comment": t.get("comment"),
        "n": get_n_from_comment(t.get("comment")),
        "transfer_id": t.get("id"),
        "amount": t.get("amount"),
        "currency": t.get("currency"),
        "status": t.get("status"),
        "transaction_date": t.get("transaction_date"),
        "post_date": t.get("post_date"),
        "reference_id": t.get("reference_id"),
        "tracking_key": t.get("tracking_key"),
        "receipt_url": t.get("receipt_url"),
    } for t in dedup])

    # orden por n
    df = df.sort_values(["n"], na_position="last").reset_index(drop=True)

    return df


In [103]:
print("RUN_ID:", RUN_ID)
prefix = f"fintoc_case_run_{RUN_ID}_batch_"
print("prefix:", prefix)

all_transfers = list_all_transfers()
print("total listados:", len(all_transfers))

# Mostrar 15 comments que existan (para ver el formato real)
comments = [t.get("comment") for t in all_transfers if isinstance(t, dict) and t.get("comment")]
print("ejemplos comments:")
for c in comments[:15]:
    print("-", c)

# Cuántos empiezan con el prefix esperado
hits = [t for t in all_transfers if str(t.get("comment","")).startswith(prefix)]
print("hits con este prefix:", len(hits))



RUN_ID: 1768267142
prefix: fintoc_case_run_1768267142_batch_
total listados: 294
ejemplos comments:
- fintoc_case_batch_72_1768267027
- fintoc_case_batch_71_1768267027
- fintoc_case_batch_70_1768267026
- fintoc_case_batch_69_1768267026
- fintoc_case_batch_68_1768267026
- fintoc_case_batch_67_1768267025
- fintoc_case_batch_66_1768267025
- fintoc_case_batch_65_1768267025
- fintoc_case_batch_64_1768267024
- fintoc_case_batch_63_1768267024
- fintoc_case_batch_62_1768267024
- fintoc_case_batch_61_1768267023
- fintoc_case_batch_60_1768267023
- fintoc_case_batch_59_1768267023
- fintoc_case_batch_58_1768267022
hits con este prefix: 0


In [104]:
import re

all_transfers = list_all_transfers()
pattern = re.compile(r"^fintoc_case_run_(\d+)_batch_(\d+)$")

runs = {}
for t in all_transfers:
    c = str(t.get("comment",""))
    m = pattern.match(c)
    if m:
        rid = m.group(1)
        runs[rid] = runs.get(rid, 0) + 1

top = sorted(runs.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top RUN_IDs:")
for rid, cnt in top:
    print(rid, "->", cnt)

# Si hay alguno con >=72, setea el primero automáticamente
candidates = [rid for rid, cnt in top if cnt >= 72]
if candidates:
    RUN_ID = candidates[0]
    print("RUN_ID seteado automáticamente a:", RUN_ID)
else:
    print("No encontré ningún RUN_ID con >=72 usando este patrón.")


Top RUN_IDs:
1768262415 -> 73
RUN_ID seteado automáticamente a: 1768262415


In [105]:
RUN_ID = "1768262415"
print("RUN_ID fijo:", RUN_ID)


RUN_ID fijo: 1768262415


In [106]:
# Polling final:
# Traigo transfers con paginación, filtro por RUN_ID, dedupeo por comment
# y espero hasta que no haya pending. Luego guardo el CSV final del run.

import time
import pandas as pd

EXPECTED = 72
POLL_EVERY_SEC = 3
MAX_WAIT_SEC = 180
FINAL_STATUSES = {"succeeded", "failed", "rejected", "returned", "canceled"}

prefix = f"fintoc_case_run_{RUN_ID}_batch_"

def get_n_from_comment(c):
    try:
        return int(str(c).split("_batch_")[1])
    except:
        return None

def pick_best_row(rows):
    # Si hay duplicados para el mismo comment, me quedo con el mejor:
    # - priorizo succeeded
    # - si empatan, uso el más "reciente" según post_date/transaction_date
    if not rows:
        return None

    def score(r):
        st = str(r.get("status", "")).lower()
        status_score = 1 if st == "succeeded" else 0
        post = r.get("post_date") or ""
        tx = r.get("transaction_date") or ""
        date_key = (post, tx)
        return (status_score, date_key)

    return max(rows, key=score)

def build_df_for_run(all_transfers, prefix):
    mine = [
        t for t in all_transfers
        if isinstance(t, dict) and str(t.get("comment", "")).startswith(prefix)
    ]

    # Agrupar por comment (1 comment = 1 batch)
    by_comment = {}
    for t in mine:
        c = t.get("comment")
        if not c:
            continue
        by_comment.setdefault(c, []).append(t)

    # Dedupe real: 1 fila por comment
    dedup = []
    for c, rows in by_comment.items():
        best = pick_best_row(rows)
        if best:
            dedup.append(best)

    df = pd.DataFrame([{
        "comment": t.get("comment"),
        "n": get_n_from_comment(t.get("comment")),
        "transfer_id": t.get("id"),
        "amount": t.get("amount"),
        "currency": t.get("currency"),
        "status": t.get("status"),
        "transaction_date": t.get("transaction_date"),
        "post_date": t.get("post_date"),
        "reference_id": t.get("reference_id"),
        "tracking_key": t.get("tracking_key"),
        "receipt_url": t.get("receipt_url"),
    } for t in dedup])

    if len(df) > 0:
        df = df.sort_values(["n"], na_position="last").reset_index(drop=True)

    return df

t0 = time.time()
attempt = 0

while True:
    attempt += 1

    all_transfers = list_all_transfers()
    df_run = build_df_for_run(all_transfers, prefix)

    found = len(df_run)

    if found > 0 and "status" in df_run.columns:
        df_run["status_norm"] = df_run["status"].astype(str).str.lower()
        pending_count = (df_run["status_norm"] == "pending").sum()
        final_count = df_run["status_norm"].isin(FINAL_STATUSES).sum()
        uniq_comments = df_run["comment"].nunique()
    else:
        pending_count = None
        final_count = None
        uniq_comments = 0

    elapsed = int(time.time() - t0)
    print(f"[poll {attempt}] rows={found} unique_comments={uniq_comments} pending={pending_count} final={final_count} elapsed={elapsed}s")

    if found == EXPECTED and pending_count == 0:
        break

    if time.time() - t0 >= MAX_WAIT_SEC:
        print("Timeout: no todas llegaron a estado final dentro del tiempo máximo.")
        break

    time.sleep(POLL_EVERY_SEC)

# limpiar columna auxiliar si existe
if "status_norm" in df_run.columns:
    df_run = df_run.drop(columns=["status_norm"])

final_path = f"transfers_report_final_run_{RUN_ID}.csv"
df_run.to_csv(final_path, index=False)

print("Final CSV:", final_path)
print("Final shape:", df_run.shape)

if "amount" in df_run.columns:
    df_run["amount"] = pd.to_numeric(df_run["amount"], errors="coerce")
    print("Suma montos (final):", int(df_run["amount"].sum()))

if "status" in df_run.columns:
    print("Status counts:")
    print(df_run["status"].value_counts(dropna=False))

df_run.head(10)


[poll 1] rows=72 unique_comments=72 pending=0 final=72 elapsed=1s
Final CSV: transfers_report_final_run_1768262415.csv
Final shape: (72, 11)
Suma montos (final): 500000000
Status counts:
status
succeeded    72
Name: count, dtype: int64


,comment,n,transfer_id,amount,currency,status,transaction_date,post_date,reference_id,tracking_key,receipt_url
0,fintoc_case_run_1768262415_batch_1,1,tr_38BBAH5smVf1c5tgQDfohxugKjx,7000000,CLP,succeeded,2026-01-13T00:09:05Z,2026-01-13T00:00:00Z,None,None,None
1,fintoc_case_run_1768262415_batch_2,2,tr_38BBAMhBgZT9v5HOdCdrEamqKIn,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None
2,fintoc_case_run_1768262415_batch_3,3,tr_38BBAOOh9MftbQEcp88zxnqlW2l,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None
3,fintoc_case_run_1768262415_batch_4,4,tr_38BBAS1EV9gMw0dq52Q2JCXkQss,7000000,CLP,succeeded,2026-01-13T00:09:05Z,2026-01-13T00:00:00Z,None,None,None
4,fintoc_case_run_1768262415_batch_5,5,tr_38BBAWcOKGDgMGBdkhYVZcgpIZT,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None
5,fintoc_case_run_1768262415_batch_6,6,tr_38BBAYp7HcljBeyQQb5Hxl36AxZ,7000000,CLP,succeeded,2026-01-13T00:09:14Z,2026-01-13T00:00:00Z,None,None,None
6,fintoc_case_run_1768262415_batch_7,7,tr_38BBAZYaPVGtA1PCjVcmf8WaGqZ,7000000,CLP,succeeded,2026-01-13T00:09:04Z,2026-01-13T00:00:00Z,None,None,None
7,fintoc_case_run_1768262415_batch_8,8,tr_38BBAhWWY4mhUQZDji5ZKktpPdZ,7000000,CLP,succeeded,2026-01-13T00:09:25Z,2026-01-13T00:00:00Z,None,None,None
8,fintoc_case_run_1768262415_batch_9,9,tr_38BBAcFALR8jDqDUa5BFQpV5iXW,7000000,CLP,succeeded,2026-01-13T00:09:05Z,2026-01-13T00:00:00Z,None,None,None
9,fintoc_case_run_1768262415_batch_10,10,tr_38BBAfibFF7TuMpXapgCCetw3gI,7000000,CLP,succeeded,2026-01-13T00:09:14Z,2026-01-13T00:00:00Z,None,None,None


In [107]:
# Validaciones duras para cerrar el caso:
# que sean 72 exactas
# que sumen 500M
# que no falte ningún batch (Que cada una de las transferencias que debían hacerse, efectivamente existe y está registrada.)
# que todas estén succeeded
# y además genero un CSV limpio + un summary.json


import json
import pandas as pd

EXPECTED = 72

# 1) Validaciones duras
df = df_run.copy()

# asegura tipos
df["n"] = pd.to_numeric(df["n"], errors="coerce").astype("Int64")
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

missing_n = sorted(set(range(1, EXPECTED + 1)) - set(df["n"].dropna().astype(int).tolist()))
dup_n = df["n"][df["n"].duplicated()].tolist()

if missing_n:
    raise ValueError(f"Faltan transferencias para n={missing_n}")
if dup_n:
    raise ValueError(f"Hay n duplicados: {dup_n}")

if len(df) != EXPECTED:
    raise ValueError(f"Se esperaban {EXPECTED} filas y hay {len(df)}")

total_amount = int(df["amount"].sum())
if total_amount != 500_000_000:
    raise ValueError(f"Suma incorrecta: {total_amount} (esperado 500000000)")

if not (df["status"].astype(str).str.lower() == "succeeded").all():
    bad = df[df["status"].astype(str).str.lower() != "succeeded"][["n","transfer_id","status","amount","comment"]]
    raise ValueError(f"Hay transferencias no exitosas:\n{bad.to_string(index=False)}")

print("Validación OK: 72/72, suma 500000000, todas succeeded, sin faltantes/duplicados.")

# 2) CSV final “limpio” (solo lo necesario)
df_clean = df[[
    "comment", "n", "transfer_id", "amount", "currency", "status",
    "transaction_date", "post_date"
]].sort_values("n").reset_index(drop=True)

final_path_clean = f"transfers_report_final_run_{RUN_ID}_clean.csv"
df_clean.to_csv(final_path_clean, index=False)
print("CSV limpio guardado:", final_path_clean)

# 3) Resumen (JSON) opcional
summary = {
    "run_id": RUN_ID,
    "expected_transfers": EXPECTED,
    "actual_transfers": int(len(df_clean)),
    "total_amount": total_amount,
    "currency": str(df_clean["currency"].iloc[0]) if len(df_clean) else None,
    "status_counts": df_clean["status"].value_counts(dropna=False).to_dict(),
}

summary_path = f"transfers_report_summary_run_{RUN_ID}.json"
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print("Resumen guardado:", summary_path)
summary


Validación OK: 72/72, suma 500000000, todas succeeded, sin faltantes/duplicados.
CSV limpio guardado: transfers_report_final_run_1768262415_clean.csv
Resumen guardado: transfers_report_summary_run_1768262415.json


{'run_id': '1768262415',
 'expected_transfers': 72,
 'actual_transfers': 72,
 'total_amount': 500000000,
 'currency': 'CLP',
 'status_counts': {'succeeded': 72}}

In [108]:
#Descarga el CSV final con el resultado completo del run para guardarlo localmente.
from google.colab import files
files.download(f"transfers_report_final_run_{RUN_ID}.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>